# video

> Video processing and sampling utils


In [ ]:
# | default_exp core.data.video

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | export
from __future__ import annotations

from pathlib import Path

import cv2
from tqdm.auto import tqdm

from ds_contrib.core.paths import PathLike

In [ ]:
# | export


def sample_frames_from_video(
    input_video: PathLike,
    output_frames_dir: PathLike,
    start_frame=0,
    max_frames=None,
    time_delta=5,
):
    """Sample frames from a video and save them to a directory.

    Simple sampler based on time delta between frames.

    Parameters
    ----------
    input_video : PathLike
        path to the video file
    output_frames_dir : PathLike
        path to the directory where frames will be saved
        NOTE: directory will be created if it doesn't exist, frames will be written to the subdirectory "part_{i:04d}"
        because batching is used, by default if the batch size is None all the frames will be saved to the same directory
    start_frame : int, optional
        from which frame to start sampling, by default 0
    max_frames : _type_, optional
        maximum number of frames to sample, by default None
    time_delta : int, optional
        time delta between frames in seconds, by default 5

    Raises
    ------
    ValueError
        if batch index (number of batches) is too large, max is 9999. Split video to shorter clips.
    """
    # Open the video file
    output_frames_dir = Path(output_frames_dir)
    output_frames_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(input_video))
    fps = cap.get(cv2.CAP_PROP_FPS)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    max_frames = max_frames or num_frames
    # get resolution
    frame_delta = int(fps * time_delta)
    for frame_ind in tqdm(
        range(start_frame, num_frames, frame_delta), desc="Sampling frames", leave=False
    ):
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_ind)
        # Read the frame
        ret, frame = cap.read()
        if not ret:
            break
        batch_ind = frame_ind // (max_frames * frame_delta)
        if batch_ind > 9999:
            raise ValueError(
                f"Batch index {batch_ind} is too large, max is 9999. Split video to shorter clips."
            )
        # Save the frame
        batch_dir = output_frames_dir / f"part_{batch_ind:04d}"
        batch_dir.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(
            str(batch_dir / f"{frame_ind:08d}.jpg"),
            frame,
            [cv2.IMWRITE_JPEG_QUALITY, 100],
        )

    cap.release()

---


#| hide

Utility section


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()